In [2]:
from confluent_kafka import Consumer, KafkaError
from datetime import datetime, timedelta
import json  # Assuming the message is in JSON format

conf = {
    'bootstrap.servers': 'b-1.monstercluster1.6xql65.c3.kafka.eu-west-2.amazonaws.com:9092',  
    'group.id': 'your_group_id',
    'auto.offset.reset': 'earliest',
}

consumer = Consumer(conf)

def consume_message():
    consumer.subscribe(['monster-damage'])
    while True:
        msg = consumer.poll(1.0)
        if msg is None or msg.error():
            if msg and msg.error().code() == KafkaError._PARTITION_EOF:
                continue
            else:
                break
        else:
            yield msg.value().decode('utf-8')

try:

    current_window_start = datetime.now()
    current_window_end = current_window_start + timedelta(minutes=10)
    window_data = []

    for message in consume_message():
        data = json.loads(message)
        print("Data", data)
        timestamp = datetime.strptime(data['ts'], '%Y-%m-%d %H:%M:%S.%f')
        
        if current_window_start <= timestamp < current_window_end:
            window_data.append(data)
        else:
            continent_counts = {}
            for entry in window_data:
                continent = entry['continent']
                continent_counts[continent] = continent_counts.get(continent, 0) + 1

            print(f"Monsters from {current_window_start} to {current_window_end}:")
            for continent, count in continent_counts.items():
                print(f"{continent}: {count} monsters")

            current_window_start = current_window_end
            current_window_end = current_window_start + timedelta(minutes=10)
            window_data = [data]

finally:
    consumer.close()

%5|1702552906.390|REQTMOUT|rdkafka#consumer-2| [thrd:GroupCoordinator]: GroupCoordinator/3: Timed out LeaveGroupRequest in flight (after 5004ms, timeout #0): possibly held back by preceeding blocking JoinGroupRequest with timeout in 297004ms
%4|1702552906.390|REQTMOUT|rdkafka#consumer-2| [thrd:GroupCoordinator]: GroupCoordinator/3: Timed out 1 in-flight, 0 retry-queued, 0 out-queue, 0 partially-sent requests
%3|1702552906.390|FAIL|rdkafka#consumer-2| [thrd:GroupCoordinator]: GroupCoordinator: b-3.monstercluster1.6xql65.c3.kafka.eu-west-2.amazonaws.com:9092: 1 request(s) timed out: disconnect (after 5997ms in state UP)
